In [60]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.integrate import quad
import matplotlib.pyplot as plt

class Flow:
    def __init__(self, mass_flow_rate: float, mass_fractions: np.ndarray,
                 temperature: float, pressure: float) -> None:
        self.mass_flow_rate = mass_flow_rate
        #self.mole_flow_rate = mole_flow_rate
        
        self.mass_fractions = mass_fractions
        #self.mole_fractions = mole_fractions
        #self.volume_fractions = volume_fractions

        #self.density = density
        #self.average_mol_mass = average_mol_mass
        #self.cp = cp
        
        self.temperature = temperature
        self.pressure = pressure
    
    def mass_to_mole_fractions(self, mr: np.ndarray) -> np.ndarray:
        mole_f = self.mass_fractions / mr
        s = mole_f.sum()
        self.mole_fractions = mole_f/s
        return self.mole_fractions
    
    def mass_to_volume_fractions(self, densities: np.ndarray) -> np.ndarray:
        vol_f = self.mass_fractions / densities
        s = vol_f.sum()
        self.volume_fractions = vol_f/s
        return self.volume_fractions
    
    def get_flow_density(self, densities: np.ndarray) -> float:
        self.density = (self.mass_fractions/densities).sum() ** -1
        return self.density
    
    def get_average_molar_mass(self, mr: np.ndarray) -> float:
        self.average_mol_mass = (self.mass_fractions / mr).sum() ** -1
        return self.average_mol_mass
    
    def get_average_cp(self, k_t: np.ndarray) -> float:
        self.cp = 0
        
        for i in range(len(self.mass_fractions)):
            cp_ = 0
            for j in range(5):
                cp_ += (j + 1) * k_t[i][j] * self.temperature ** j
            self.cp += cp_ * self.mass_fractions[i]
        
        '''
        self.cp = np.sum(
            np.sum([(j + 1) * k_t[i][j] * self.temperature ** (j+1-1) for j in range(5)])
            * self.mass_fractions[i] for i in range(len(self.mass_fractions))
        )
        '''
        return self.cp

class Mixer:
    def mix(self, *flows: Flow) -> Flow:
        self.flows = flows
    
        mass_flow_rate = np.sum([flow.mass_flow_rate for flow in self.flows])
        
        mass_fractions = np.divide(np.sum(
            [flow.mass_fractions * flow.mass_flow_rate for flow in self.flows],
            axis = 0), mass_flow_rate)
        
        cp_flow = np.sum([flow.cp * flow.mass_flow_rate for flow in self.flows]
                        ) / mass_flow_rate

        temp_flow = np.sum([flow.cp * flow.mass_flow_rate * flow.temperature
                            for flow in self.flows]) / (mass_flow_rate * cp_flow)

        pressure = min(flow.pressure for flow in self.flows)

        self.mixture = Flow(mass_flow_rate=mass_flow_rate,
                           mass_fractions = mass_fractions,
                           temperature = temp_flow,
                           pressure = pressure)
        return self.mixture
    
class Heatexchanger:
    def __init__(self, d_in: float = .1,
                 d_out: float = .25, length: float = 3.0, k: float = 4900
                ) -> None:
        self.d_in = d_in
        self.d_out = d_out
        self.length = length
        self.k = k
        
    # объемная скорость = масс. расход*4/(плотность*пи*диаметр**2)
    def calculate(self, hot: Flow, cold: Flow) -> tuple[Flow]:
        
        v_c = (cold.mass_flow_rate/cold.density) / (3.14 * self.d_in **2 / 4 * self.length)
        v_h = (hot.mass_flow_rate/hot.density) / (3.14 * self.d_out ** 2 / 4 * self.length 
                                        - 3.14 * self.d_in **2 / 4 * self.length)
        
        
        def func_hot(length):
            dTh = -self.k*np.pi*self.d_out/((hot.mass_flow_rate * 4/np.pi/self.d_out**2) 
                                * hot.cp) * (hot.temperature - cold.temperature) * length
            return dTh
        def func_cold(length):
            dTc = self.k*np.pi*self.d_in/((cold.mass_flow_rate * 4/np.pi/self.d_in**2) 
                                * cold.cp) * (hot.temperature - cold.temperature) * length
            return dTc

        Th = quad(func_hot, 0, self.length)
        Tc = quad(func_cold, 0, self.length)
        
        th = hot.temperature - Th[0]
        tc = cold.temperature - Tc[0]
       
        leng = 0
        h = 0.01
        '''
        Th_, Tc_ = hot.temperature, cold.temperature
        while leng <= self.length:
            Th_ -= self.k*np.pi*self.d_out/(v_h * hot.density * hot.cp) *(hot.temperature - cold.temperature) * h
            Tc_ += self.k*np.pi*self.d_in/(v_h * hot.density * cold.cp) *(hot.temperature - cold.temperature) * h
            leng += h
        '''
        hot_ = Flow(mass_flow_rate=hot.mass_flow_rate,
                           mass_fractions = hot.mass_fractions,
                           temperature = th,
                           pressure = hot.pressure)
        cold_ = Flow(mass_flow_rate=cold.mass_flow_rate,
                           mass_fractions = cold.mass_fractions,
                           temperature = tc,
                           pressure = cold.pressure)        
        return [hot_, cold_]
    
class Splitter:
    def calculate(self, flow: Flow, *ratio: float) -> list[Flow]:
        #mass_flow_rates = [flow.mass_flow_rate * rat for rat in ratio]
        result = []
        for rat in ratio:
            split_ = Flow(mass_flow_rate = flow.mass_flow_rate * rat,
                           mass_fractions = flow.mass_fractions,
                           temperature = flow.temperature,
                           pressure = flow.pressure)
            result.append(split_)
        return result

if __name__ == '__main__':
    densities = np.array([0.821537454674234, 8.57E-05, 0.634118153548788, 0.0138331933625558])
    mr = np.array([128.1332, 2.02, 131.82935, 34.01])
    mf1 = np.random.randint(1, 5 , 4)
    #mass_fraction = mf1 / mf1.sum()
    mass_fraction = np.array([0.42, 0.12, 0.26, 0.2])
    
    k_ = [[0.071254, 0.002979, -0.0000007, 0, 0],
         [13.83761, 0.0003, 0.000000346, -0.000000000097, 0.00000000000000773],
         [-0.09689, 0.003473, -0.0000013, 0.000000000256, -0.000000000000014],
         [0.9985, -0.00018, 0.000000557, -0.00000000032, 0.0000000000000637]]
    
    
    f1 = Flow(mass_flow_rate=1098, mass_fractions=mass_fraction,
    temperature=300, pressure=101.325)
    
    f1.volume_fractions = f1.mass_to_volume_fractions(densities)
    f1.mole_fractions = f1.mass_to_mole_fractions(mr)
    f1.density = f1.get_flow_density(densities)
    f1.mol_mass = f1.get_average_molar_mass(mr)
    f1.cp = f1.get_average_cp(k_)
    
    mf2 = np.random.randint(1, 5, 4)
    #mass_fraction2 = mf2 / mf2.sum()
    mass_fraction2 = np.array([0.35, 0.20, 0.42, 0.03])
    
    f2 = Flow(mass_flow_rate=2000, mass_fractions=mass_fraction2, temperature=314, pressure=101.325)
    f2.cp = f2.get_average_cp(k_)

    f2.volume_fractions = f2.mass_to_volume_fractions(densities)
    f2.mole_fractions = f2.mass_to_mole_fractions(mr)
    f2.density = f2.get_flow_density(densities)
    f2.mol_mass = f2.get_average_molar_mass(mr)
    f2.cp = f2.get_average_cp(k_)
    
    mf3 = np.random.randint(1, 5 , 4)
    #mass_fraction3 = mf3 / mf3.sum()
    mass_fraction3 = np.array([0.23, 0.25, 0.10, 0.32])
    
    f3 = Flow(mass_flow_rate=1635, mass_fractions=mass_fraction3, temperature=267, pressure=101.325)
    f3.cp = f3.get_average_cp(k_)
    
    f3.volume_fractions = f3.mass_to_volume_fractions(densities)
    f3.mole_fractions = f3.mass_to_mole_fractions(mr)
    f3.density = f3.get_flow_density(densities)
    f3.mol_mass = f3.get_average_molar_mass(mr)
    f3.cp = f3.get_average_cp(k_)

    m = Mixer()
    f4 = m.mix(f1, f2, f3)

    f4.density = f4.get_flow_density(densities)
    f4.mol_mass = f4.get_average_molar_mass(mr)
    f4.cp = f4.get_average_cp(k_)

    mf8 = np.random.randint(1, 5 , 4)
    #mass_fraction8 = mf8 / mf8.sum()
    mass_fraction8 = np.array([0.12, 0.45, 0.25, 0.18])
    
    f8 = Flow(mass_flow_rate=2000, mass_fractions=mass_fraction8, temperature=500, pressure=101.325)

    f8.density = f8.get_flow_density(densities)
    f8.mol_mass = f8.get_average_molar_mass(mr)
    f8.cp = f8.get_average_cp(k_)
    
    HE = Heatexchanger()
    f5, f9 = HE.calculate(f4, f8)
    
    f5.density = f5.get_flow_density(densities)
    f5.mol_mass = f5.get_average_molar_mass(mr)
    f5.cp = f5.get_average_cp(k_)

    
    f9.density = f9.get_flow_density(densities)
    f9.mol_mass = f9.get_average_molar_mass(mr)
    f9.cp = f9.get_average_cp(k_)
    
    split = Splitter()
    f6, f7 = split.calculate(f5, 0.3, 0.7)
    
    f6.density = f6.get_flow_density(densities)
    f6.mol_mass = f6.get_average_molar_mass(mr)
    f6.cp = f6.get_average_cp(k_)
 
    f7.density = f6.get_flow_density(densities)
    f7.mol_mass = f6.get_average_molar_mass(mr)
    f7.cp = f6.get_average_cp(k_)
    
    flows = [f1, f2, f3, f4, f5, f6, f7, f8, f9]
    name = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    for i in range(9):
            print('Поток', name[i])
            print('Расход', flows[i].mass_flow_rate, 'кг/ч')
            print('Температура', flows[i].temperature, '\u00b0С')
            print('Состав, % масс.', flows[i].mass_fractions, '')
            print('Плотность потока',flows[i].density, 'г/см\u00b3')
            print('Молярная масса', flows[i].mol_mass, 'кг/кмоль')
            print('Теплоемкость', flows[i].cp, 'кДж/кг')
            print()

Поток 1
Расход 1098 кг/ч
Температура 300 °С
Состав, % масс. [0.42 0.12 0.26 0.2 ] 
Плотность потока 0.0007064079513799702 г/см³
Молярная масса 14.177027193278898 кг/кмоль
Теплоемкость 3.0275183578 кДж/кг

Поток 2
Расход 2000 кг/ч
Температура 314 °С
Состав, % масс. [0.35 0.2  0.42 0.03] 
Плотность потока 0.00042790279659089586 г/см³
Молярная масса 9.450950440133632 кг/кмоль
Теплоемкость 4.1878822950382375 кДж/кг

Поток 3
Расход 1635 кг/ч
Температура 267 °С
Состав, % масс. [0.23 0.25 0.1  0.32] 
Плотность потока 0.00034005239630385353 г/см³
Молярная масса 7.367842796516575 кг/кмоль
Теплоемкость 4.33347424486937 кДж/кг

Поток 4
Расход 4733 кг/ч
Температура 293.7955499156533 °С
Состав, % масс. [0.32478555 0.19871329 0.2723389  0.16961758] 
Плотность потока 0.0004288542367639002 г/см³
Молярная масса 9.262621701711254 кг/кмоль
Теплоемкость 3.948850634350893 кДж/кг

Поток 5
Расход 4733 кг/ч
Температура 284.4164924645595 °С
Состав, % масс. [0.32478555 0.19871329 0.2723389  0.16961758] 
Плотнос

In [ ]:
from scipy.integrate import quad
import numpy as np

k = 4900
d_out, d_in = 0.25, 0.1
hot_cp, cold_cp = 5, 7
hot_mass_flow_rate, cold_mass_flow_rate = 200, 150
hot_temperature, cold_temperature = 100, 200

def funh(length):
    dTh = -k*np.pi*d_out/((hot_mass_flow_rate * 4/np.pi/d_out**2) 
                        * hot_cp) * (hot_temperature - cold_temperature) * length
    return dTh

def func(length):
    dTc = k*np.pi*d_in/((cold_mass_flow_rate * 4/np.pi/d_in**2) * cold_cp) * (hot_temperature - cold_temperature) * length
    return dTc

length = 3
Th = quad(funh, 0, length)
Tc = quad(func, 0, length)
print(Th[0])
print(Tc[0])

leng = 0
h = 1

Th_, Tc_ = 0, 0
while leng <= length:
    Th_ -= k * np.pi * d_out / ((hot_mass_flow_rate * 4/np.pi/d_out**2) * hot_cp) * (hot_temperature - cold_temperature) * h
    Tc_ += k * np.pi * d_in / ((cold_mass_flow_rate * 4/np.pi/d_in**2) * cold_cp) * (hot_temperature - cold_temperature) * h
    leng += h
print(Th_, Tc_)